# Partition

In [ ]:
from qualtran import Bloq, CompositeBloq, BloqBuilder, Signature, Register
from qualtran import QBit, QInt, QUInt, QAny
from qualtran.drawing import show_bloq, show_call_graph, show_counts_sigma
from typing import *
import numpy as np
import sympy
import cirq

## `Partition`
Partition a generic index into multiple registers.

#### Parameters
 - `n`: The total bit-size of the un-partitioned register. Required if `dtype_in` is None. Deprecated. Kept for backward compatibility. Use `dtype_in` instead whenever possible.
 - `regs`: Registers to partition into. The `side` attribute is ignored.
 - `dtype_in`: Type of the un-partitioned register. Required if `n` is None. If None, the type is inferred as `QUInt(n)`.
 - `partition`: `False` means un-partition instead. 

#### Registers
 - `x`: the un-partitioned register. LEFT by default.
 - `[user spec]`: The registers provided by the `regs` argument. RIGHT by default.


In [ ]:
from qualtran.bloqs.bookkeeping import Partition

### Example Instances

In [ ]:
regs = (Register('xx', QAny(2), shape=(2, 3)), Register('yy', QAny(37)))
bitsize = sum(reg.total_bits() for reg in regs)
partition = Partition(n=bitsize, regs=regs)

#### Graphical Signature

In [ ]:
from qualtran.drawing import show_bloqs
show_bloqs([partition],
           ['`partition`'])

## `AutoPartition`
Automatically adds and undoes `Partition` of registers to match the signature of a sub-bloq.

This tool enables using a bloq in a context expecting an alternative signature that combines
registers in the bloq's signature or operates over more registers than the bloq does.
For example, it can adapt a bloq exposing multiple selection registers to a quantum interface
that expects only one unified selection register.

Wrapping in `AutoPartition` also hides splits and joins behind a level of decomposition, which
can produce more helpful circuit diagrams compared to manually splitting and joining.

#### Parameters
 - `bloq`: The sub-bloq to wrap. Its register names are used within the second items in each pair in the `partitions` argument below.
 - `partitions`: A sequence of pairs specifying each register that is exposed in the external signature of the `AutoPartition` and its relationship to the registers of `bloq`. The first element of each pair is a `Register` exposed externally. The second is a list of register names of `bloq` that concatenate to form the externally exposed register. If `bloq` does not operate on some portion (of `n` bits) of the externally exposed register, the sentinel value `Unused(n)` can be used in place of a register name.
 - `left_only`: If False, the output registers will also follow `partition`. Otherwise, the output registers will follow `bloq.signature.rights()`. This flag must be set to True if `bloq` does not have the same LEFT and RIGHT registers, as is required for the bloq to be fully wrapped on the left and right. 

#### Registers
 - `[user_spec]`: The output registers of the wrapped bloq.


In [ ]:
from qualtran.bloqs.bookkeeping import AutoPartition

### Example Instances

In [ ]:
from qualtran import Controlled, CtrlSpec
from qualtran.bloqs.basic_gates import Swap

bloq = Controlled(Swap(1), CtrlSpec())
auto_partition = AutoPartition(
    bloq, [(Register('x', QAny(2)), ['ctrl', 'x']), (Register('y', QAny(1)), ['y'])]
)

In [ ]:
from qualtran import Controlled, CtrlSpec
from qualtran.bloqs.basic_gates import Swap
from qualtran.bloqs.bookkeeping.auto_partition import Unused

bloq = Controlled(Swap(1), CtrlSpec())
auto_partition_unused = AutoPartition(
    bloq,
    [
        (Register('x', QAny(3)), ['ctrl', 'x', Unused(1)]),
        (Register('y', QAny(1)), ['y']),
        (Register('z', QAny(2)), [Unused(2)]),
    ],
)

#### Graphical Signature

In [ ]:
from qualtran.drawing import show_bloqs
show_bloqs([auto_partition, auto_partition_unused],
           ['`auto_partition`', '`auto_partition_unused`'])

### Call Graph

In [ ]:
from qualtran.resource_counting.generalizers import ignore_split_join
auto_partition_g, auto_partition_sigma = auto_partition.call_graph(max_depth=1, generalizer=ignore_split_join)
show_call_graph(auto_partition_g)
show_counts_sigma(auto_partition_sigma)